In [34]:
# import packages
import geopandas as gpd
import osmnx as ox
import momepy
import pandana as pdna
import pandas as pd
import numpy as np
import networkx as nx

from scipy.spatial import cKDTree
from shapely.geometry import Point

from shapely.geometry import box

In [35]:
## load data

# load geopackages
pois = gpd.read_file('C:/Users/b9066009/Documents/PhD/conferences/2022_GISRUK/gisruk_2022/pois_buffer.gpkg')
postcodes = gpd.read_file('C:/Users/b9066009/Documents/PhD/conferences/2022_GISRUK/gisruk_2022/codepoints.gpkg')

# load streets from OSM
walk_graph = ox.graph_from_bbox(north=53.5093, south=53.3186,east=-2.8015,west=-3.0638, network_type = 'walk') # download walking network

In [36]:
## clean data

# streets
walk_graph = ox.get_undirected(walk_graph) # cleans the network keeping parallel edges only if geometry is different
walk_graph = ox.projection.project_graph(walk_graph) # project graph
walk_streets = ox.graph_to_gdfs(walk_graph, nodes=False, edges=True, node_geometry=False, fill_edge_geometry=True) # convert graph to gdf
walk_streets = walk_streets[["geometry", "from", "to", "length"]] # clean columns
walk_streets = walk_streets.to_crs(27700) # set crs

# postcodes
postcodes = postcodes.reset_index(drop=True).explode().reset_index(drop=True) # clean
postcodes['postcodeID'] = range(0,len(postcodes)) # generate index column
postcodes = postcodes.set_index('postcodeID') # set index

# pois
pois = pois.reset_index(drop=True).explode().reset_index(drop=True) # avoid multipart pois

amenities = ['Bakeries','Butchers','Confectioners','Delicatessens','Fishmongers','Grocers, Farm Shops and Pick Your Own','Convenience Stores and Independent Supermarkets','Supermarket Chains',
             'Bus Stops','Railway Stations, Junctions and Halts',
             'Gymnasiums, Sports Halls and Leisure Centres','Sport and Entertainment'
             'Higher Education Establishments','First, Primary and Infant Schools','Broad Age Range and Secondary State Schools',
             'Doctors Surgeries','Hospitals','Chemists and Pharmacies',
             'Cinemas','Theatres and Concert Halls','Art Galleries'] # select which amenities to keep
pois_filtered = pois[pois.classname.isin(amenities)] # filter amenities
pois_filtered = pois_filtered[['groupname','categoryname','classname','geometry']] # clean columns
pois_filtered['poiID'] = range(0,len(pois_filtered)) # generate index column
pois_filtered = pois_filtered.set_index('poiID') # set index column

C:\Users\b9066009\AppData\Local\Temp\ipykernel_14764\1478089765.py:11: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  postcodes = postcodes.reset_index(drop=True).explode().reset_index(drop=True) # clean
C:\Users\b9066009\AppData\Local\Temp\ipykernel_14764\1478089765.py:16: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  pois = pois.reset_index(drop=True).explode().reset_index(drop=True) # avoid multipart pois


In [37]:
## generate pandana network
# nodes and edges for walk network
nodes_walk, edges_walk = momepy.nx_to_gdf( # convert network to gdf
    momepy.gdf_to_nx( # convert to nx graph
        walk_streets.explode() # remove multipart rows
    )
)
nodes_walk = nodes_walk.set_index('nodeID') # set index

# generate walk pandana network
walk_streets_pdna = pdna.Network( 
    nodes_walk.geometry.x,
    nodes_walk.geometry.y,
    edges_walk['node_start'], # set origins
    edges_walk['node_end'], # set destinations
    edges_walk[['mm_len']] # set edge length
)

C:\Users\b9066009\AppData\Local\Temp\ipykernel_14764\1581756382.py:5: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  walk_streets.explode() # remove multipart rows


In [38]:
# attach pois to the network
walk_streets_pdna.set_pois( # snap pois to network
    category = 'pois', # set name of the new layer snapped on the network
    maxdist = 1200, # set maximum distance
    maxitems = 700, # set maximum number of pois to look for
    x_col = pois_filtered.geometry.x,
    y_col = pois_filtered.geometry.y
)

In [39]:
results = walk_streets_pdna.nearest_pois( # calculate distances to pois
    distance = 1200, # maximum distance
    category = 'pois', # layer where we want to look for
    num_pois = 700, # max number of pois to look for
    include_poi_ids = True # include pois ids
)

In [40]:
# store results separately as distances and poiIDs

# separate distances from poi ids
distances = results.iloc[:,:round(len(results.columns)/2,)] # create df with distances
pois_ids = results.iloc[:,round(len(results.columns)/2,):] # create df with pois ids

# convert wide matrices to long
distances_long = pd.melt(distances.reset_index(), id_vars='nodeID',value_name='distance') # make matrix long
pois_ids_long = pd.melt(pois_ids.reset_index(), id_vars='nodeID',value_name='poiID') # make matrix long

# create an od long df containing nodeID, distance, and poiID
od = distances_long
od['poiID'] = pois_ids_long['poiID'].astype('Int64') # set a column with pois ids (as they are indexed, they are already in the right order)

# format od matrix and drop NAs
od = od[['nodeID','poiID','distance']] # clean columns
od = od.dropna() # drop NAs 

In [41]:
# merge od data with POIs data
pois_filtered = pois_filtered.reset_index() # reset index pois df
od_pois_info = pd.merge(od, pois_filtered[['groupname','categoryname','classname','poiID']].reset_index(), left_on='poiID', right_on='poiID') # merge pois info to od matrix

In [42]:
# add postcode information to the od_pois_info 
postcodes_nodes = walk_streets_pdna.get_node_ids( # get nearest street nodes to each postcode
    postcodes.geometry.x,
    postcodes.geometry.y
)
postcodes_nodes = pd.DataFrame(postcodes_nodes).reset_index() # reset index
postcodes_nodes = postcodes_nodes.rename(columns={'node_id':'nodeID'}) # change col names

# add postcodes to od_pois_info
od_pois_info = pd.merge(od_pois_info, postcodes_nodes, left_on='nodeID', right_on='nodeID') # add postcodes to od matrix

In [43]:
od_pois_info.to_csv('C:/Users/b9066009/Documents/PhD/conferences/2022_GISRUK/gisruk_2022/od_pois_buffer.csv', index=False)